In [42]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [65]:
def parse_results_file(fname, labels):
  with open(fname, "r") as f:
    data = f.read()
  df_data = {
    "time": []
  }
  for label in labels:
    df_data[label] = []

  header_is_next = False
  header_read = False
  for line in data.split("\n"):
    if not line:
      continue
    if header_is_next:
      header = line
      header_read = True
      header_is_next = False
    elif "*" in line:
      header_is_next = not header_read
    else:
      parts = [part.strip() for part in line.split("|")]
      for part in parts:
        x_label = part.split("=")[0]
        if x_label in labels:
          x_val = float(part.split("=")[-1])
          df_data[x_label].append(x_val)
      y_val = float(parts[-1])
      df_data['time'].append(y_val)
  return header, pd.DataFrame(data=df_data)

def plot(files, x_label, names=[]):
  fig = make_subplots()
  fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
  ))
  for i, fname in enumerate(files):
    header, df = parse_results_file(fname, [x_label])
    trace = go.Scatter(x=df[x_label], y=df["time"], mode='lines', name=names[i] if len(names) > i else header)
    fig.add_trace(trace, secondary_y=False)
  return fig

In [67]:
plot([
  "ompscaling/20240509_225938_2thin_ompscaling.txt",
  "ompscaling/20240509_233604_2thin_ompscaling.txt"
], "places", [
  "OMP_PROC_BIND=close",
  "OMP_PROC_BIND=spread"
])

In [69]:
plot([
  "mpiscaling/20240509_214003_2thin_mpiscaling.txt"
], "nodes")

In [71]:
plot([
  "weak/20240512_164030_thin_weak.txt"
], "ny")

In [72]:
plot([
  "corescaling/20240510_145158_2epyc_corescaling.txt"
], "cores")